# Cal-CRAI Metric Calculation for: 
This notebook calculates one metric sourced from the Environmental Protection Agency

* Percentage of days with unhealthy or worse air quality days per county between 1980-2022

In [1]:
import pandas as pd
import os
import sys

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/society_economy/social_services/census_bureau_soc_services/listed_soc_services/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'multiple_social_services_2021_CB2100CBP.csv'
Saved DataFrame as 'multiple_social_services_2021_CB2100CBP_metadata.csv'


In [3]:
# read in social service data
social_services_data = pd.read_csv('multiple_social_services_2021_CB2100CBP.csv')
print(len(social_services_data))
social_services_data.head(5)
os.remove('multiple_social_services_2021_CB2100CBP.csv')

1067


In [4]:
# drop duplicates, original dataset had a repeat year
# filtered_air_quality = air_quality.drop_duplicates(subset=['Year', 'County'])
social_services_data.head(5)

,GEO_ID,NAME,GEO_ID_F,NAICS2017,NAICS2017_F,NAICS2017_LABEL,LFO,LFO_LABEL,EMPSZES,EMPSZES_LABEL,...,PAYANN_N_F,PAYQTR1,PAYQTR1_F,PAYQTR1_N,PAYQTR1_N_F,EMP,EMP_F,EMP_N,EMP_N_F,Unnamed: 25
0,Geographic identifier code,Geographic Area Name,Geo Footnote,2017 NAICS code,2017 NAICS Footnote,Meaning of NAICS Code,Legal form of organization code,Meaning of Legal form of organization code,Employment size of establishments code,Meaning of Employment size of establishments code,...,Flag for Noise range for annual payroll,"First-quarter payroll ($1,000)",Flag for first-quarter payroll,Noise range for first-quarter payroll,Flag for Noise range for first-quarter payroll,Number of employees,Flag for number of employees,Noise range for number of employees,Flag for Noise range for number of employees,NaN
1,0500000US06001,"Alameda County, California",NaN,237,NaN,Heavy and civil engineering construction,001,All establishments,001,All establishments,...,H,165683,NaN,H,H,6088,NaN,H,H,NaN
2,0500000US06001,"Alameda County, California",NaN,237,NaN,Heavy and civil engineering construction,001,All establishments,210,Establishments with less than 5 employees,...,N,N,N,N,N,N,N,N,N,NaN
3,0500000US06001,"Alameda County, California",NaN,237,NaN,Heavy and civil engineering construction,001,All establishments,220,Establishments with 5 to 9 employees,...,N,N,N,N,N,N,N,N,N,NaN
4,0500000US06001,"Alameda County, California",NaN,237,NaN,Heavy and civil engineering construction,001,All establishments,230,Establishments with 10 to 19 employees,...,N,N,N,N,N,N,N,N,N,NaN


In [5]:
# Set the first row as new column names
new_headers = social_services_data.iloc[0]

# Replace the headers with the first row values
social_services_data = social_services_data.rename(columns=new_headers)

# Remove the first row (if needed)
social_services_data = social_services_data.drop(0)

In [6]:
social_services_columns = social_services_data[['Geographic Area Name', 'Meaning of NAICS Code', 'Meaning of Employment size of establishments code', 'Number of establishments']]
social_services_columns

,Geographic Area Name,Meaning of NAICS Code,Meaning of Employment size of establishments code,Number of establishments
1,"Alameda County, California",Heavy and civil engineering construction,All establishments,113
2,"Alameda County, California",Heavy and civil engineering construction,Establishments with less than 5 employees,35
3,"Alameda County, California",Heavy and civil engineering construction,Establishments with 5 to 9 employees,13
4,"Alameda County, California",Heavy and civil engineering construction,Establishments with 10 to 19 employees,19
5,"Alameda County, California",Heavy and civil engineering construction,Establishments with 20 to 49 employees,21
...,...,...,...,...
1062,"Yuba County, California",Specialty trade contractors,Establishments with less than 5 employees,47
1063,"Yuba County, California",Specialty trade contractors,Establishments with 5 to 9 employees,12
1064,"Yuba County, California",Specialty trade contractors,Establishments with 10 to 19 employees,8
1065,"Yuba County, California",Health and personal care stores,All establishments,8


In [7]:
# Create a copy of the DataFrame
social_services_columns = social_services_columns.copy()

# Create a new column 'county' by extracting characters after the first '-'
social_services_columns['county'] = social_services_columns['Geographic Area Name'].str.split(',').str[0]

# Remove any mention of 'county' within the legalAgencyName column
social_services_columns['county'] = social_services_columns['county'].str.replace(' County', '', case=False)

# Drop the 'Geographic Area Name' column
social_services_county = social_services_columns.drop('Geographic Area Name', axis=1)

social_services_county.head(5)

,Meaning of NAICS Code,Meaning of Employment size of establishments code,Number of establishments,county
1,Heavy and civil engineering construction,All establishments,113,Alameda
2,Heavy and civil engineering construction,Establishments with less than 5 employees,35,Alameda
3,Heavy and civil engineering construction,Establishments with 5 to 9 employees,13,Alameda
4,Heavy and civil engineering construction,Establishments with 10 to 19 employees,19,Alameda
5,Heavy and civil engineering construction,Establishments with 20 to 49 employees,21,Alameda


In [37]:
social_services_county = social_services_county[social_services_columns['Meaning of Employment size of establishments code'] == 'All establishments'].copy()
social_services_county.head()

C:\Users\jespi\AppData\Local\Temp\ipykernel_33364\4070930651.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  social_services_county = social_services_county[social_services_columns['Meaning of Employment size of establishments code'] == 'All establishments'].copy()


,Meaning of NAICS Code,Meaning of Employment size of establishments code,Number of establishments,county
1,Heavy and civil engineering construction,All establishments,113,Alameda
8,Specialty trade contractors,All establishments,1793,Alameda
17,Health and personal care stores,All establishments,370,Alameda
23,Blood and organ banks,All establishments,7,Alameda
24,Hospitals,All establishments,42,Alameda


In [38]:
filtered_social_services, omitted_df = filter_counties(social_services_county, 'county')

In [32]:
print(len(filtered_social_services))
print(len(omitted_df))

251
0


In [39]:
filtered_social_services

,Meaning of NAICS Code,Meaning of Employment size of establishments code,Number of establishments,county
1,Heavy and civil engineering construction,All establishments,113,Alameda
8,Specialty trade contractors,All establishments,1793,Alameda
17,Health and personal care stores,All establishments,370,Alameda
23,Blood and organ banks,All establishments,7,Alameda
24,Hospitals,All establishments,42,Alameda
...,...,...,...,...
1056,Hospitals,All establishments,3,Yolo
1057,Personal and household goods repair and mainte...,All establishments,8,Yolo
1059,Heavy and civil engineering construction,All establishments,10,Yuba
1061,Specialty trade contractors,All establishments,68,Yuba


In [40]:
further_filtered_social_services = filtered_social_services.drop(columns='Meaning of Employment size of establishments code')
further_filtered_social_services

,Meaning of NAICS Code,Number of establishments,county
1,Heavy and civil engineering construction,113,Alameda
8,Specialty trade contractors,1793,Alameda
17,Health and personal care stores,370,Alameda
23,Blood and organ banks,7,Alameda
24,Hospitals,42,Alameda
...,...,...,...
1056,Hospitals,3,Yolo
1057,Personal and household goods repair and mainte...,8,Yolo
1059,Heavy and civil engineering construction,10,Yuba
1061,Specialty trade contractors,68,Yuba


In [51]:
adjusted_social_services = further_filtered_social_services.pivot(index='county', columns='Meaning of NAICS Code', values='Number of establishments')
# Reset index to make 'county' a column again
adjusted_social_services = adjusted_social_services.reset_index()

# Flatten the multi-level header by setting the columns
adjusted_social_services.columns.name = None  # Remove the multi-level header name
adjusted_social_services.columns = [str(col).lower() for col in adjusted_social_services.columns]  # Flatten the header

adjusted_social_services.head()

,county,blood and organ banks,health and personal care stores,heavy and civil engineering construction,hospitals,personal and household goods repair and maintenance,specialty trade contractors
0,Alameda,7,370,113,42,84,1793
1,Amador,NaN,4,9,NaN,3,59
2,Butte,NaN,52,22,5,4,353
3,Calaveras,NaN,7,9,NaN,5,113
4,Colusa,NaN,3,4,NaN,NaN,22


## Our metrics are per 10,000 people, so we need to add our population data
* these metrics are at the county level so we need need county level population estimates

In [24]:
county_pop = "s3://ca-climate-index/0_map_data/county_est_pop_2021.csv"
ca_pop_county = pd.read_csv(county_pop)
ca_pop_county = ca_pop_county.drop(columns='Unnamed: 0')
ca_pop_county.head()

,county,est_total_pop
0,Alameda,1663823
1,Alpine,1515
2,Amador,40577
3,Butte,213605
4,Calaveras,45674


In [54]:
merged_county_social_services = pd.merge(ca_pop_county, adjusted_social_services, on='county', how='right')
merged_county_social_services.head()

,county,est_total_pop,blood and organ banks,health and personal care stores,heavy and civil engineering construction,hospitals,personal and household goods repair and maintenance,specialty trade contractors
0,Alameda,1663823,7,370,113,42,84,1793
1,Amador,40577,NaN,4,9,NaN,3,59
2,Butte,213605,NaN,52,22,5,4,353
3,Calaveras,45674,NaN,7,9,NaN,5,113
4,Colusa,21811,NaN,3,4,NaN,NaN,22


In [56]:
# Create a new DataFrame with the same 'county' and 'est_total_pop' columns
social_services_per_10000 = merged_county_social_services[['county', 'est_total_pop']].copy()

# Iterate over the columns to rename them and calculate the new values
for column in merged_county_social_services.columns:
    merged_county_social_services[column] = pd.to_numeric(merged_county_social_services[column], errors='coerce')
    if column not in ['county', 'est_total_pop']:
        # Rename the column
        new_column_name = f"{column} per 10000 people"
        # Calculate the new values
        social_services_per_10000[new_column_name] = (merged_county_social_services[column] / merged_county_social_services['est_total_pop']) * 10000

social_services_per_10000

,county,est_total_pop,blood and organ banks per 10000 people,health and personal care stores per 10000 people,heavy and civil engineering construction per 10000 people,hospitals per 10000 people,personal and household goods repair and maintenance per 10000 people,specialty trade contractors per 10000 people
0,Alameda,1663823,0.042072,2.223794,0.679159,0.252431,0.504861,10.776387
1,Amador,40577,NaN,0.985780,2.218005,NaN,0.739335,14.540257
2,Butte,213605,NaN,2.434400,1.029938,0.234077,0.187262,16.525830
3,Calaveras,45674,NaN,1.532601,1.970486,NaN,1.094715,24.740553
4,Colusa,21811,NaN,1.375453,1.833937,NaN,NaN,10.086654
5,Contra Costa,1162648,NaN,1.840626,0.903111,0.197824,0.498861,13.125211
6,Del Norte,27462,NaN,NaN,NaN,1.456558,NaN,7.646930
7,El Dorado,191713,NaN,1.669162,1.721323,0.156484,0.573774,23.159619
8,Fresno,1008280,0.079343,1.983576,0.882691,0.148768,0.277701,10.969175
9,Glenn,28657,NaN,2.093729,1.046865,NaN,NaN,15.005060


In [25]:
est_pop = "s3://ca-climate-index/0_map_data/cri_acs_demographic_estimated_population.csv"
ca_est_pop = pd.read_csv(est_pop)
ca_est_pop = ca_est_pop.rename(columns={'Census_Tract': 'TRACT'})
ca_est_pop = ca_est_pop.drop(columns='Unnamed: 0')
ca_est_pop

,census_tract,est_total_pop
0,6001400100,3269
1,6001400200,2147
2,6001400300,5619
3,6001400400,4278
4,6001400500,3949
...,...,...
9124,6115040902,1868
9125,6115041001,3672
9126,6115041002,3417
9127,6115041101,2288


In [ ]:
county_est_pop_2021['est_total_pop'] = county_est_pop_2021['est_total_pop'].astype(float)

In [ ]:
from pandas.api.types import is_float_dtype

In [ ]:
is_float_dtype(county_est_pop_2021['est_total_pop'])

True

In [ ]:
county_est_pop_2021.to_csv('county_est_pop_2021.csv')

## Uploading our new estimated population per county csv data

In [ ]:
county_est_pop_2021.to_csv('county_est_pop_2021.csv')
bucket_name = 'ca-climate-index'
directory = '0_map_data'
upload_csv_aws('county_est_pop_2021.csv', bucket_name, directory)

FileNotFoundError: [Errno 2] No such file or directory: 'c'

## Function call

In [ ]:
import numpy as np
import pandas as pd

def make_social_service_metrics(df, export=False): 
    # Read in social service data
    df = pd.read_csv('multiple_social_services_2021_CB2100CBP.csv', header=1)
    
    # # Set the first row as new column names
    # new_headers = df.iloc[0]

    # # Replace the headers with the first row values
    # df = df.rename(columns=new_headers)

    # # Remove the first row (if needed)
    # df = df.drop(0)
        
    social_services_columns = df[['Geographic Area Name', 'Meaning of NAICS Code', 'Meaning of Employment size of establishments code', 'Number of establishments']]
    
    # Create a copy of the DataFrame
    social_services_columns = social_services_columns.copy()

    # Create a new column 'county' by extracting characters after the first '-'
    social_services_columns['county'] = social_services_columns['Geographic Area Name'].str.split(',').str[0]

    # Remove any mention of 'county' within the legalAgencyName column
    social_services_columns['county'] = social_services_columns['county'].str.replace(' County', '', case=False)

    # Drop the 'Geographic Area Name' column
    social_services_county = social_services_columns.drop('Geographic Area Name', axis=1)
    
    social_services_county = social_services_county[social_services_columns['Meaning of Employment size of establishments code'] == 'All establishments'].copy()

    filtered_social_services, omitted_df = filter_counties(social_services_county, 'county')
      
    # Get unique values in the column
    unique_values = filtered_social_services['Meaning of NAICS Code'].unique()

    selected_columns = ['county', 'Meaning of NAICS Code', 'Number of establishments']

    # Create a dictionary to store DataFrames
    dfs = {}

    # Iterate over unique values and create a new DataFrame for each
    for value in unique_values:
        # Filter rows where the value matches and where 'Meaning of Employment size of establishments code' is 'All establishments'
        df_filtered = social_services_columns[(social_services_columns['Meaning of NAICS Code'] == value) & 
                                            (social_services_columns['Meaning of Employment size of establishments code'] == 'All establishments')][selected_columns].copy()
        
        # Add to the dictionary if any rows are filtered
        if not df_filtered.empty:
            dfs[value] = df_filtered

    # Merge with population data
    tracts_county = "s3://ca-climate-index/0_map_data/county_est_pop_2021.csv"
    ca_tracts_county = pd.read_csv(tracts_county)
    for key, df in dfs.items():
        # Merge the dataframe with population data
        df = pd.merge(df, ca_tracts_county[['county', 'est_total_pop']], on='county', how='left')
        # Update the original dictionary value with the modified dataframe
        dfs[key] = df
    
    # Process each dataframe
    for key, df in dfs.items():
        # Convert 'est_total_pop' column to numeric
        df['est_total_pop'] = df['est_total_pop'].astype(float)
        # df['est_total_pop'] = pd.to_numeric(df['est_total_pop'], errors='coerce')
        # Filter out non-numeric and zero values
        df = df[(pd.notnull(df['est_total_pop'])) & (df['est_total_pop'] != 0)]
        # Check if 'est_total_pop' column contains any non-numeric values
        # if df['est_total_pop'].dtype != np.number:
        if is_float_dtype(county_est_pop_2021['est_total_pop']) == False:
            print(f"Warning: Non-numeric values found in 'est_total_pop' column for {key}. Skipping calculations.")
            continue
        # Create a new column for each dataframe
        new_column_name = 'establishments_per_10000_people'
        df[new_column_name] = (df['Number of establishments'] / df['est_total_pop']) * 10000
        # Update the original dictionary value with the modified dataframe
        dfs[key] = df

    # Export data if export is True
    if export:
        for key, df in dfs.items():
            filename = f'society_social_services_{key.replace(" ", "_")}_metric.csv'
            print("Filename:", filename)  # Print the filename for debugging purposes
            df.to_csv(filename)
            
            # Define AWS upload parameters
            bucket_name = 'ca-climate-index'
            directory = '3_fair_data/index_data'
            
            # Upload to AWS
            upload_csv_aws(filename, bucket_name, directory)
            
            # Remove local file
            #if os.path.exists(filename):
            #    os.remove(filename)

# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/society_economy/social_services/census_bureau_soc_services/listed_soc_services/'
pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

# read in social service data
social_services_data = pd.read_csv('multiple_social_services_2021_CB2100CBP.csv')
make_social_service_metrics(social_services_data, export=True)

Saved DataFrame as 'multiple_social_services_2021_CB2100CBP.csv'
Saved DataFrame as 'multiple_social_services_2021_CB2100CBP_metadata.csv'
Filename: society_social_services_Heavy_and_civil_engineering_construction_metric.csv


FileNotFoundError: [Errno 2] No such file or directory: 's'

In [ ]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/society_economy/social_services/census_bureau_soc_services/listed_soc_services/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

# read in social service data
social_services_data = pd.read_csv('multiple_social_services_2021_CB2100CBP.csv')
make_social_service_metrics(social_services_data, export=True)

Saved DataFrame as 'multiple_social_services_2021_CB2100CBP.csv'
Saved DataFrame as 'multiple_social_services_2021_CB2100CBP_metadata.csv'


C:\Users\jesse\AppData\Local\Temp\ipykernel_10388\1081700643.py:68: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df['est_total_pop'].dtype != np.number:
C:\Users\jesse\AppData\Local\Temp\ipykernel_10388\1081700643.py:68: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df['est_total_pop'].dtype != np.number:
C:\Users\jesse\AppData\Local\Temp\ipykernel_10388\1081700643.py:68: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df['est_total_pop'].dtype != np.number:
C:\Users\jesse\AppData\Local\Temp\ipykernel_10388\1081700643.py:68: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly co

Filename: society_social_services_Heavy_and_civil_engineering_construction_metric.csv
society_social_services_heavy_and_civil_engineering_construction_metric.csv uploaded to AWS
Filename: society_social_services_Specialty_trade_contractors_metric.csv
society_social_services_specialty_trade_contractors_metric.csv uploaded to AWS
Filename: society_social_services_Health_and_personal_care_stores_metric.csv
society_social_services_health_and_personal_care_stores_metric.csv uploaded to AWS
Filename: society_social_services_Blood_and_organ_banks_metric.csv
society_social_services_blood_and_organ_banks_metric.csv uploaded to AWS
Filename: society_social_services_Hospitals_metric.csv
society_social_services_hospitals_metric.csv uploaded to AWS
Filename: society_social_services_Personal_and_household_goods_repair_and_maintenance_metric.csv
society_social_services_personal_and_household_goods_repair_and_maintenance_metric.csv uploaded to AWS
